# Import e Open

## Lembretes

PRIORIDADES
* Quantos jogadores tinha na mesa no começo da mão
* Em que posição eu tava
------
BUGS ETC
* Ver o negócio de alguns torneios não estarem puxando o hero_placement
* Tirar a coluna "match_id" da tabela "silver_dim_seats" porque tá bagunçando a modelagem. fazer o mesmo na fact_actions
* mudar o id da partida pra incluir a sigla do site antes

------
* dim_matches: hero_placement tem algumas linhas vazias, provavelmente porque deu alguma pala na hora de salvar o arquivo e a mão em que eu perdi o jogo não ficou salva. ver como lidar

## Imports

In [ ]:
## Imports

import pandas as pd
import numpy as np
import json
import pprint
import os
import re

## Testes JSON

In [ ]:
JSON_FOLDER = r"/content/drive/MyDrive/Projetos/poker_app/json_files/"

json_files_list = os.listdir(JSON_FOLDER)

with open(f"{JSON_FOLDER}{json_files_list[19]}") as test_json_file:
  test_new_dict = json.loads(json.load(test_json_file))
  #pprint.pprint(test_new_dict, sort_dicts = False)

#json_files_list#[19]

## Abertura de arquivos

In [ ]:
SILVER_FOLDER = r"/content/drive/MyDrive/Projetos/poker_app/silver_csv_files/"

json_files_list = os.listdir(SILVER_FOLDER)
#print(json_files_list)

silver_matches_file = f"{SILVER_FOLDER}silver_dim_matches.csv"
silver_seats_file = f"{SILVER_FOLDER}silver_dim_seats.csv"
silver_hands_file = f"{SILVER_FOLDER}silver_dim_hands.csv"
silver_actions_file = f"{SILVER_FOLDER}silver_fact_actions.csv"

df_silver_matches = pd.read_csv(silver_matches_file, sep=";", parse_dates=["start_datetime"], infer_datetime_format=True)
df_silver_seats = pd.read_csv(silver_seats_file, sep=";")
df_silver_hands = pd.read_csv(silver_hands_file, sep=";", parse_dates=["start_datetime"], infer_datetime_format=True)
df_silver_actions = pd.read_csv(silver_actions_file, sep=";")

In [ ]:
# algumas transformações básicas

df_silver_matches['bet_limit'] = df_silver_matches['bet_limit'].replace(['No', 'Yes'], [False, True])
df_silver_seats['starts_sitting_out'] = df_silver_seats['starts_sitting_out'].fillna(('No')).replace(['No', 'Yes'], [False, True])
df_silver_actions['amount'] = df_silver_actions['amount'].fillna(0).astype("int")
df_silver_actions['new_amount'] = df_silver_actions['new_amount'].fillna(0).astype("int")
df_silver_actions['all_in'] = df_silver_actions['all_in'].fillna(('N')).replace(['N', 'Y'], [False, True])

# Métodos e Constantes

## Constantes

In [ ]:
ROMAN_TO_ARABIC = {
    'I': 1,
    'II': 2,
    'III': 3,
    'IV': 4,
    'V': 5,
    'VI': 6,
    'VII': 7,
    'VIII': 8,
    'IX': 9,
    'X': 10,
    'XI': 11,
    'XII': 12,
    'XIII': 13
}

In [ ]:
CARD_HIERARCHY = {'A': 13, 'K': 12, 'Q': 11, 'J': 10, 'T': 9, '9': 8, '8': 7, '7': 6, '6': 5, '5': 4, '4': 3, '3': 2, '2': 1}

In [ ]:
GENERAL_EP_HANDS = ['AA', 'AKs', 'AQs', 'AJs', 'ATs',
                    'AKo', 'KK', 'KQs', 'KJs',
                    'AQo', 'KQo', 'QQ', 'QJs',
                    'AJo', 'JJ',
                    'ATo', 'TT']

GENERAL_MP_HANDS = ['AA', 'AKs', 'AQs', 'AJs', 'ATs', 'A9s', 'A8s', 'A7s', 'A6s', 'A5s', 'A4s', 'A3s', 'A2s',
                    'AKo', 'KK', 'KQs', 'KJs', 'KTs',
                    'AQo', 'KQo', 'QQ', 'QJs', 'QTs',
                    'AJo', 'KJo', 'QJo', 'JJ', 'JTs', 'J9s',
                    'ATo', 'KTo', 'QTo', 'JTo', 'TT', 'T9s',
                    '99', '98s',
                    '88', '87s',
                    '77',
                    '66']

GENERAL_LP_HANDS = ['AA', 'AKs', 'AQs', 'AJs', 'ATs', 'A9s', 'A8s', 'A7s', 'A6s', 'A5s', 'A4s', 'A3s', 'A2s',
                    'AKo', 'KK', 'KQs', 'KJs', 'KTs', 'K9s', 'K8s', 'K7s', 'K6s', 'K5s', 'K4s', 'K3s', 'K2s',
                    'AQo', 'KQo', 'QQ', 'QJs', 'QTs', 'Q9s', 'Q8s',
                    'AJo', 'KJo', 'QJo', 'JJ', 'JTs', 'J9s', 'J8s',
                    'ATo', 'KTo', 'QTo', 'JTo', 'TT', 'T9s', 'T8s',
                    'A9o', 'K9o', 'Q9o', 'J9o', 'T9o', '99', '98s', '97s',
                    'A8o', 'K8o', 'Q8o', 'J8o', 'T8o', '98o', '88', '87s', '86s',
                    'A7o', 'K7o', '77', '76s', '75s',
                    'A6o', 'K6o', '66', '65s',
                    'A5o', 'K5o', '55', '54s',
                    'A4o', '44',
                    'A3o', '33',
                    'A2o', '22']

In [ ]:
# não tenho lembrança de usar isso em nenhum lugar. talvez apagar?
TABLE_POSITIONS = {'9': 'BU',
                   '8': 'CO',
                   '7': 'MP3',
                   '6': 'MP2',
                   '5': 'MP1',
                   '4': 'UTG+1',
                   '3': 'UTG',
                   '2': 'BB',
                   '1': 'SB'}

In [ ]:
PLAY_ORDER = {
    0: 'UTG',
    1: 'UTG+1',
    2: 'UTG+2',
    3: 'MP1',
    4: 'MP2',
    5: 'CO',
    6: 'BU',
    7: 'SB',
    8: 'BB',
}

## Métodos pras transformações

In [ ]:
# transforma os níveis das mãos de algarismo romano pra arábico
# pra dim_gold_playedhands

def convert_roman_to_arabic(roman_number):
  arabic_number = 0
  arabic_number = ROMAN_TO_ARABIC[roman_number]
  return arabic_number

#convert_roman_to_arabic('VII')

In [ ]:
# converte as duas cartas da mão no short. Ex: AhTc -> ATo
# pra dim_gold_playedhands

def convert_hand_to_short(hand):

  card1_letter = hand[0]
  card2_letter = hand[2]
  card1_suit = hand[1]
  card2_suit = hand[3]

  ordered_hand_letters = ""
  suitedness = ""

  if CARD_HIERARCHY[card1_letter] < CARD_HIERARCHY[card2_letter]:
    ordered_hand_letters = f"{card2_letter}{card1_letter}"
  else:
    ordered_hand_letters = f"{card1_letter}{card2_letter}"

  if card1_suit == card2_suit:
    suitedness = "s"
  else:
    suitedness = "o"

  if card1_letter == card2_letter:
    suitedness = ""

  short_hand = f"{ordered_hand_letters}{suitedness}"

  return short_hand


#convert_hand_to_short('KsKc')

## Métodos pra formatar tipo de coluna

In [ ]:
def float_to_string(df, columns):
    for col in columns:
        df[col] = (
            df[col]
            .astype('string')
            .str.replace('.',',')
        )

    return None

# Tabelas Gold

## gold_dim_finishers

In [ ]:
finishers = df_silver_actions.query("action_description.str.contains('finishes')")[['match_id', 'player', 'action_description']]
finishers['placement'] = finishers['action_description'].str.split(pat=' ', n=1, expand=True)[1].str.extract('(\d+)')
dim_finishers = finishers[['match_id', 'placement', 'player']].sort_values(['match_id', 'placement'], ascending=[True, True])

In [ ]:
# teste da tabela acima

dim_finishers

,match_id,placement,player
876,3491017877,3,oextraordimario
806,3491017877,4,Eagle I
739,3491017877,5,lmbbsa
352,3491017877,6,williamrenaud
322,3491017877,7,sanguesuga77777
...,...,...,...
72430,3687289002,5,pedropaluu
71786,3687289002,6,Trenchopen
71640,3687289002,7,repolhallin
71438,3687289002,8,ig_valmor


## gold_dim_matches

In [ ]:
df_silver_matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              90 non-null     int64         
 1   platform        90 non-null     object        
 2   match_type      90 non-null     object        
 3   buy_in          90 non-null     float64       
 4   currency        90 non-null     object        
 5   poker_type      90 non-null     object        
 6   bet_limit       90 non-null     bool          
 7   start_datetime  90 non-null     datetime64[ns]
 8   timezone        90 non-null     object        
dtypes: bool(1), datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 5.8+ KB


In [ ]:
# puxar dados de outras tabelas e transformar em DFs

time_last_hand = df_silver_hands.groupby('match_id')[['start_datetime']].max().rename(columns={'start_datetime': 'lasthand_start_datetime'})
hero_placement = dim_finishers.query("player == 'oextraordimario'")[['match_id', 'placement']].rename(columns={'placement': 'hero_placement'}).set_index('match_id')
hands_seen = df_silver_hands.groupby('match_id')[['id']].nunique().rename(columns={'id': 'hands_seen'})
flops_opened = df_silver_actions.query("stage == 'flop' and player == 'oextraordimario'").groupby('match_id')[['hand_id']].nunique().rename(columns={'hand_id': 'flops_opened'})
times_active_preflop = df_silver_actions.query("stage == 'pre_flop' and player == 'oextraordimario' and action_description != 'folds'").groupby('match_id')[['hand_id']].nunique().rename(columns={'hand_id': 'times_active_preflop'})
times_shoved_preflop = df_silver_actions.query("stage == 'pre_flop' and player == 'oextraordimario' and all_in == True").groupby('match_id')[['hand_id']].nunique().rename(columns={'hand_id': 'times_shoved_preflop'})

In [ ]:
# importações mais complexas

# número de vezes em que eu shovei preflop, alguém(s) pagou e fui direto pro showdown
hands_shoved_preflop = df_silver_actions.query("stage == 'pre_flop' and player == 'oextraordimario' and all_in == True")['hand_id'] #.groupby('match_id')[['hand_id']].nunique()
mask = df_silver_actions['hand_id'].isin(hands_shoved_preflop)
pf_shove_to_showdown = df_silver_actions.loc[mask].query("stage == 'showdown'").groupby('match_id')[['hand_id']].nunique().rename(columns={'hand_id': 'pf_shove_to_showdown'})

In [ ]:
# join de colunas vindas de outras tabelas

dim_matches = (df_silver_matches
 .merge(time_last_hand, how='left', left_on='id', right_on='match_id')
 .merge(hero_placement, how='left', left_on='id', right_on='match_id')
 .merge(flops_opened, how='left', left_on='id', right_on='match_id')
 .merge(times_shoved_preflop, how='left', left_on='id', right_on='match_id')
 .merge(pf_shove_to_showdown, how='left', left_on='id', right_on='match_id')
 .merge(times_active_preflop, how='left', left_on='id', right_on='match_id')
 .merge(hands_seen, how='left', left_on='id', right_on='match_id')
 )

dim_matches = dim_matches.assign(match_duration = dim_matches['lasthand_start_datetime'] - dim_matches['start_datetime'])
dim_matches['match_duration'] = dim_matches['match_duration'].dt.total_seconds().astype('int64')
dim_matches['flops_opened'] = dim_matches['flops_opened'].fillna(0).astype('int64')
dim_matches['times_shoved_preflop'] = dim_matches['times_shoved_preflop'].fillna(0).astype('int64')
dim_matches['pf_shove_to_showdown'] = dim_matches['pf_shove_to_showdown'].fillna(0).astype('int64')

In [ ]:
# gambiarra

dim_matches['hero_placement'] = dim_matches['hero_placement'].fillna(-1).astype('int64')

In [ ]:
dim_matches.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90 entries, 0 to 89
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   id                       90 non-null     int64         
 1   platform                 90 non-null     object        
 2   match_type               90 non-null     object        
 3   buy_in                   90 non-null     float64       
 4   currency                 90 non-null     object        
 5   poker_type               90 non-null     object        
 6   bet_limit                90 non-null     bool          
 7   start_datetime           90 non-null     datetime64[ns]
 8   timezone                 90 non-null     object        
 9   lasthand_start_datetime  90 non-null     datetime64[ns]
 10  hero_placement           90 non-null     int64         
 11  flops_opened             90 non-null     int64         
 12  times_shoved_preflop     90 non-null  

## gold_dim_seats

In [ ]:
df_silver_seats.head()

#df_silver_seats.assign(cumulativecount = df_silver_seats.groupby('hand_id').cumcount() + 1)

,match_id,hand_id,seat_number,seat_player,seat_chips,starts_sitting_out
0,3491017877,239454475952,1,raymond49,1500,False
1,3491017877,239454475952,2,queiroz2610,1500,False
2,3491017877,239454475952,3,Eagle I,1500,False
3,3491017877,239454475952,4,ksei,1500,False
4,3491017877,239454475952,5,sanguesuga77777,1500,False


### Cálculo da posição

In [ ]:
join_button_seats = df_silver_hands[['id', 'button_seat']].assign(position = 'BU')

In [ ]:
join_seat_positions1 = (
    df_silver_seats
    .merge(
        join_button_seats,
        how='left',
        left_on=['hand_id', 'seat_number'],
        right_on=['id', 'button_seat'])
    [['hand_id', 'seat_number', 'seat_player', 'position']]
).assign(helper_number = df_silver_seats['seat_number'])

In [ ]:
helper_list = []
last_hand = 0
switch = False

for index, row in join_seat_positions1.iterrows():
  new_helper = row['helper_number']
  current_hand = row['hand_id']

  if current_hand != last_hand:
    switch = False

  if switch == True:
    new_helper -= 10

  if row['position'] == 'BU':
    switch = True

  #print(f"{last_hand}, {current_hand}")
  helper_list.append(new_helper)

  last_hand = current_hand

In [ ]:
join_seat_positions2 = (
    join_seat_positions1
    .assign(helper_number = helper_list)
    .sort_values(by=['hand_id', 'helper_number'], ascending = [True, False])
)

join_seat_positions2['play_order'] = join_seat_positions2.groupby('hand_id').cumcount() - 3

In [ ]:
# Update 'position' based on 'play_order' values

def update_position(row):
    if row['play_order'] == -2:
        return 'SB'
    elif row['play_order'] == -1:
        return 'BB'
    else:
        return row['position']

join_seat_positions2['position'] = join_seat_positions2.apply(update_position, axis=1)

In [ ]:
join_seat_positions2['current_players'] = join_seat_positions2.groupby('hand_id')['seat_player'].transform('nunique')
join_seat_positions2['new_play_order'] = join_seat_positions2['play_order'] + (9 - join_seat_positions2['current_players'])

In [ ]:
def update_position2(row):
  if pd.isna(row['position']):
    pos = PLAY_ORDER[row['new_play_order']]
  else:
    pos = row['position']
  return pos

join_seat_positions2['position'] = join_seat_positions2.apply(update_position2, axis=1)

In [ ]:
join_seat_positions2.head(-10)

,hand_id,seat_number,seat_player,position,helper_number,play_order,current_players,new_play_order
0,239454475952,1,raymond49,BU,1,-3,9,-3
8,239454475952,9,williamrenaud,SB,-1,-2,9,-2
7,239454475952,8,lmbbsa,BB,-2,-1,9,-1
6,239454475952,7,oextraordimario,UTG,-3,0,9,0
5,239454475952,6,shirl263,UTG+1,-4,1,9,1
...,...,...,...,...,...,...,...,...
27053,247695171173,4,oextraordimario,SB,4,-2,2,5
27055,247695173039,4,oextraordimario,BU,4,-3,2,4
27056,247695173039,8,Shadøw73,SB,-2,-2,2,5
27058,247695175851,8,Shadøw73,BU,8,-3,2,4


In [ ]:
def update_position3(row):
  pos = row['position']

  if row['current_players'] == 2:
    if row['position'] == 'BU':
      pos = 'BU_HU'
    elif row['position'] == 'SB':
      pos = 'BU_BB'

  return pos

join_seat_positions2['position'] = join_seat_positions2.apply(update_position3, axis=1)

In [ ]:
join_seat_positions3 = join_seat_positions2[['hand_id', 'seat_number', 'position']]

In [ ]:
join_seat_positions3

,hand_id,seat_number,position
0,239454475952,1,BU
8,239454475952,9,SB
7,239454475952,8,BB
6,239454475952,7,UTG
5,239454475952,6,UTG+1
...,...,...,...
27064,247695184298,8,BU_BB
27066,247695189016,8,BU_HU
27065,247695189016,4,BU_BB
27067,247695190832,4,BU_HU


### Join da posição

In [ ]:
dim_seats = df_silver_seats.merge(join_seat_positions3, how='left', left_on=['hand_id', 'seat_number'], right_on=['hand_id', 'seat_number'])

In [ ]:
dim_seats

,match_id,hand_id,seat_number,seat_player,seat_chips,starts_sitting_out,position
0,3491017877,239454475952,1,raymond49,1500,False,BU
1,3491017877,239454475952,2,queiroz2610,1500,False,CO
2,3491017877,239454475952,3,Eagle I,1500,False,MP2
3,3491017877,239454475952,4,ksei,1500,False,MP1
4,3491017877,239454475952,5,sanguesuga77777,1500,False,UTG+2
...,...,...,...,...,...,...,...
27537,3667709428,246900617229,7,Le marin 53,3026,False,BU_HU
27538,3667709428,246900618498,4,oextraordimario,10824,False,BU_HU
27539,3667709428,246900618498,7,Le marin 53,2676,False,BU_BB
27540,3667709428,246900620950,4,oextraordimario,11474,False,BU_BB


## gold_dim_playedhands

### Joins

In [ ]:
#flops_opened = df_silver_actions.query("stage == 'flop' and player == 'oextraordimario'").groupby('match_id')[['hand_id']].nunique().rename(columns={'hand_id': 'flops_opened'})
#times_active_preflop = df_silver_actions.query("stage == 'pre_flop' and player == 'oextraordimario' and action_description != 'folds'").groupby('match_id')[['hand_id']].nunique().rename(columns={'hand_id': 'times_active_preflop'})

players_on_flop = (
    df_silver_actions
    .query("stage == 'flop'")
    .groupby('hand_id')[['player']]
    .nunique()
    .rename(columns={'player': 'players_on_flop'}))

hero_active_preflop = (
    df_silver_actions
    .query("stage == 'pre_flop' and player == 'oextraordimario' and action_description not in ['folds', 'posts ante']")
    .groupby('hand_id')[['action_number']]
    .min()
    .rename(columns={'action_number': 'hero_active_preflop'}))

hero_saw_flop = (
    df_silver_actions
    .query("stage == 'flop' and player == 'oextraordimario'")
    .groupby('hand_id')[['action_number']]
    .min()
    .rename(columns={'action_number': 'hero_saw_flop'}))

players_start_hand = (
    df_silver_seats
    .groupby('hand_id')['seat_player']
    .count())

hero_position = (
    dim_seats
    .query("seat_player == 'oextraordimario'")[['hand_id', 'position']]
    .rename(columns={'position': 'hero_position'})
)

In [ ]:
df_silver_actions.query("action_description == 'folds'")

,match_id,hand_id,stage,action_number,player,action_description,amount,new_amount,all_in
2,3491017877,239454475952,pre_flop,1,ksei,folds,0,0,False
5,3491017877,239454475952,pre_flop,4,oextraordimario,folds,0,0,False
7,3491017877,239454475952,pre_flop,6,lmbbsa,folds,0,0,False
10,3491017877,239454475952,pre_flop,9,raymond49,folds,0,0,False
11,3491017877,239454475952,pre_flop,10,queiroz2610,folds,0,0,False
...,...,...,...,...,...,...,...,...,...
12979,3501153343,239885014670,pre_flop,1,Marymaud,folds,0,0,False
12981,3501153343,239885014670,pre_flop,3,DJayZee,folds,0,0,False
12982,3501153343,239885014670,pre_flop,4,Clesin,folds,0,0,False
12984,3501153343,239885014670,pre_flop,6,Lobaosp2,folds,0,0,False


In [ ]:
join_hands_seats = df_silver_seats.query("seat_player == 'oextraordimario'")[['hand_id', 'seat_chips', 'starts_sitting_out']]

join_hands_seats = join_hands_seats.rename(columns={
    'seat_chips': 'hero_starting_chips',
    'starts_sitting_out': 'hero_sitting_out'
})

In [ ]:
dim_hands = (
    df_silver_hands
    .merge(players_on_flop, how='left', left_on='id', right_on='hand_id')
    .merge(hero_active_preflop, how='left', left_on='id', right_on='hand_id')
    .merge(hero_saw_flop, how='left', left_on='id', right_on='hand_id')
    .merge(join_hands_seats, how='left', left_on='id', right_on='hand_id').drop(columns='hand_id')
    .merge(players_start_hand, how='left', left_on='id', right_on='hand_id').rename(columns={'seat_player': 'players_start_hand'})
    .merge(hero_position, how='left', left_on='id', right_on='hand_id').drop(columns='hand_id')
)

In [ ]:
dim_hands.query("id == 240289227103")

,id,match_id,table_id,current_lvl,price_sb,price_bb,start_datetime,max_players,button_seat,hero_card1,...,river_card,total_pot,rake,players_on_flop,hero_active_preflop,hero_saw_flop,hero_starting_chips,hero_sitting_out,players_start_hand,hero_position
1436,240289227103,3511320806,3511320806 1,VII,100,200,2022-12-05 20:07:56,9,5,2d,...,NaN,2315,0,NaN,2.0,NaN,4515,False,7,UTG+2


### Colunas calculadas

In [ ]:
dim_hands = (
    dim_hands
    .assign(players_on_flop = dim_hands['players_on_flop'].fillna(0).astype('int64'))
    .assign(current_lvl = dim_hands['current_lvl'].apply(convert_roman_to_arabic))
    .assign(hero_hand_short = dim_hands['hero_card1'].str.cat(dim_hands['hero_card2']).apply(convert_hand_to_short))
    .assign(hero_active_preflop = dim_hands['hero_active_preflop'] > 0)
    .assign(hero_saw_flop = dim_hands['hero_saw_flop'] > 0)
    .assign(hero_starting_blinds = round(dim_hands['hero_starting_chips'] / dim_hands['price_bb'], 1))
    .assign(total_pot_blinds = round(dim_hands['total_pot'] / dim_hands['price_bb'], 1))
)

#dim_hands['lvl_variation'] = dim_hands['current_lvl'].diff().fillna(0).astype('int64') # montei isso aqui mas real que nem é relevante
dim_hands['hero_chips_variation'] = dim_hands['hero_starting_chips'].diff().shift(-1).fillna(0).astype('int64')
dim_hands['hero_blinds_variation'] = round(dim_hands['hero_chips_variation'] / dim_hands['price_bb'], 1)

## gold_fact_handactions

In [ ]:
df_silver_actions.head()

,match_id,hand_id,stage,action_number,player,action_description,amount,new_amount,all_in
0,3491017877,239454475952,forced_bets,1,queiroz2610,posts small_blind,10,0,False
1,3491017877,239454475952,forced_bets,2,Eagle I,posts big_blind,20,0,False
2,3491017877,239454475952,pre_flop,1,ksei,folds,0,0,False
3,3491017877,239454475952,pre_flop,2,sanguesuga77777,raises,20,40,False
4,3491017877,239454475952,pre_flop,3,shirl263,calls,40,0,False


## Tratamentos de formato

In [ ]:
#float_to_string(dim_matches, ['buy_in'])

In [ ]:
#float_to_string(dim_hands, ['hero_starting_blinds', 'total_pot_blinds', 'hero_blinds_variation'])

## Savings

In [ ]:
#dim_finishers.to_csv("/content/drive/MyDrive/Projetos/poker_app/gold_csv_files/gold_dim_finishers.csv", sep=";", decimal=",", index=False)
#dim_matches.to_csv("/content/drive/MyDrive/Projetos/poker_app/gold_csv_files/gold_dim_matches.csv", sep=";", decimal=",", index=False)
dim_hands.to_csv("/content/drive/MyDrive/Projetos/poker_app/gold_csv_files/gold_dim_playedhands.csv", sep=";", decimal=",", index=False)

In [ ]:
#join_seat_positions2.to_csv("/content/drive/MyDrive/Projetos/poker_app/test_csv_files/join_seat_positions2.csv", sep=";", index=False)